<a href="https://colab.research.google.com/github/Nilllas/RL_CustomGymEnvCars/blob/main/RL_CustomGymCarsEnv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 ''' I want you to help me with writing a code example for training a reinforcement learning algorithm on a dataset that I have. The environment have different worlds that the agent can play. Each world have between 6-8 levels. the agent plays a world from first to last level. In each level the agent will pick either 1, 2 or 3 cars that it believes will win the level amongst a total of 8-15 cars. The state of the agent is a vector with features of the level and features of all cars in a particular level. The agent will get a reward if it manages to pick a car that wins the race. The reward is the product of 10 times the odds for a particular car to win. For example, in World A there are 6 levels and in level 2 there are 9 cars - the agent picks 2 cars that it believes will win the level - the agent pics Car number 5 which has odds 2.0 and Car number 8 which has odds 4.5. Car number 8 wins so the agent gets 10 times 4.5 which equals a reward of 45. If the agent manages to get a positive reward in all levels of a particular world it will get an extra world reward that is calculated as the sum of each reward of each level in that particular world. Let's see how a code example can look like. start with the agent and the state functions.

In [2]:
# import libraries

import numpy as np
import random
import pandas as pd
import json
import string


In [3]:
# create a list of random world ids

NUM_WORLDS = 3

def id_generator(size=3, chars=string.ascii_uppercase):
      return ''.join(random.choice(chars) for _ in range(size))

world_ids =[]

for _ in range(NUM_WORLDS):
    world_ids.append(id_generator())

LEVELS_PER_WORLD = [random.randrange(2,5,1) for _ in range(len(world_ids))]

print("number of worlds: "+str(NUM_WORLDS))
print("world ids: "+str(world_ids))
print("random num lev per world: "+str(LEVELS_PER_WORLD))


number of worlds: 3
world ids: ['WQG', 'XYW', 'KWG']
random num lev per world: [2, 3, 4]


In [4]:
# create synthetic data


# create a dictionary of data frames for each world
worlds = {}

# loop over each world id
for world_id in world_ids:

    # create an empty list of levels
    levels = []

    # get the number of levels for this world
    num_levels = LEVELS_PER_WORLD[world_ids.index(world_id)]
    
    print("World id: "+str(world_id),"num levels: "+str(num_levels))

    # loop over each level number
    for level in range(1, num_levels +1):

        # get the number of cars for this level
        CARS_PER_LEVEL = [random.randrange(4,9,1) for _ in range(1, num_levels +1)]
        num_cars = CARS_PER_LEVEL[level - 1]
        
        print("level: "+str(level), "num cars in level: "+str(num_cars))

        # create a random list of car numbers
        car_numbers = np.random.permutation(np.arange(1, num_cars + 1))
        #print(car_numbers)

        # create a random list of wheel types (black or blue)
        wheel_types = np.random.choice(["black", "blue", "red"], num_cars)

        # create a random list of endurance levels (high or low)
        motor_endurance = np.random.choice(["high", "medium", "low"], num_cars)

        # create a random list of odds (between 1 and 100)
        nr = np.arange(2,30)
        odds = np.random.choice(nr, num_cars)

        if not (2 in odds):
          odds[0] = 2 # replace the first element with 2
        if not (99 in odds):
          odds[-1] = 99 # replace the last element with 99
        
        # create a data frame for this level with columns: car_number, wheel_type, endurance_level and odds
        level_df = pd.DataFrame({"level_number": level,
                                 "car_number": car_numbers,
                                 "wheel_type": wheel_types,
                                 "motor_endurance": motor_endurance,
                                 "odds": odds})

        # add a column for the level distance (between 1000 and 3000)
        level_df["level_distance"] = random.randrange(1000, 3500, 500)

        # add a column for the difficulty level (easy or hard)
        level_df["level_difficulty"] = np.random.choice(["easy", "moderate", "hard"])

        # add a column for the winner (a random car number based on their odds)
        winner_probs = 1 / level_df["odds"] #/ sum(level_df["odds"])
        level_df["winner_probs"] = winner_probs
        winner_car_number = random.choices(level_df["car_number"], weights=winner_probs)[0]
        level_df["winner"] = winner_car_number

        #level_df.drop('odds', axis=1, inplace=True)

        # append this data frame to the list of levels
        levels.append(level_df)

    # create a data frame for this world with columns: world_id, level_number and cars_data (a nested data frame)
    world_df = pd.DataFrame({"world_id": [world_id] * num_levels,
                             "world_type": "V"+str(num_levels),
                             "level_number": range(1, num_levels + 1),
                             "cars_data": levels})

    # store this data frame in the dictionary of worlds with key: world_id
    worlds[world_id] = world_df

World id: WQG num levels: 2
level: 1 num cars in level: 7
level: 2 num cars in level: 5
World id: XYW num levels: 3
level: 1 num cars in level: 5
level: 2 num cars in level: 5
level: 3 num cars in level: 6
World id: KWG num levels: 4
level: 1 num cars in level: 8
level: 2 num cars in level: 5
level: 3 num cars in level: 4
level: 4 num cars in level: 4


In [5]:
# create a list of dataframes
all_data_list = []
for world_id, world_df in worlds.items():

    # iterate over the rows of world_df
    for index, row in world_df.iterrows():

        # access the cars_data column (a list of dataframes)
        level_cars_data = row["cars_data"]

        level_cars_data['world_id']=world_id
        level_cars_data['world_type']=world_df['world_type'][index]
        # add world_id, world_type and level_number columns to cars_data
        
        
        # append row_df to all_data_list
        all_data_list.append(level_cars_data)

# create a single dataframe from all_data_list
all_data = pd.concat(all_data_list)

# display the first 10 rows of the dataframe
all_data.sort_values(by=['world_id','level_number','car_number'])

,level_number,car_number,wheel_type,motor_endurance,odds,level_distance,level_difficulty,winner_probs,winner,world_id,world_type
5,1,1,blue,high,11,1000,hard,0.090909,6,KWG,V4
4,1,2,black,low,6,1000,hard,0.166667,6,KWG,V4
7,1,3,black,high,99,1000,hard,0.010101,6,KWG,V4
6,1,4,black,high,13,1000,hard,0.076923,6,KWG,V4
1,1,5,black,medium,28,1000,hard,0.035714,6,KWG,V4
3,1,6,black,high,5,1000,hard,0.200000,6,KWG,V4
2,1,7,blue,high,14,1000,hard,0.071429,6,KWG,V4
0,1,8,red,low,2,1000,hard,0.500000,6,KWG,V4
4,2,1,blue,medium,99,1500,easy,0.010101,5,KWG,V4
3,2,2,black,high,18,1500,easy,0.055556,5,KWG,V4


In [ ]:

'''
The agent: This is the entity that interacts with the environment and learns from its actions. In your case, this could be a car that races in different levels and worlds.
The state: This is the representation of the situation that the agent faces at each step. In your case, this could be a vector of features that describe the car’s position, speed, wheel type, endurance level, etc.
The action: This is the choice that the agent makes at each step to change its state. In your case, this could be accelerating, braking, turning left or right, etc.
The reward: This is the feedback that the agent receives after taking an action. It can be positive or negative depending on how well the action aligns with the goal. In your case, this could be a function of how close or far the car is from winning or losing.



In [ ]:
''' I have a dataset for a game on which I would like to train a Reinforcement Learning (RL) agent. let's name the dataset car_worlds. The agent will at each step observe a level in a world. The agent action at each step is to pick either 1, 2 or 3 cars amongst a total number of cars in that level which the agen believes will win the race. The dataset contains features for both the world with features such as a unique world_id and world_type. Additionally the dataset contains features for the levels in a particular world such as level_difficulty and level_distance as well as features for all cars in a particular level such as car_number, car_odds, and winner_probs. At each step the agent action is to pick 1, 2 or 3 cars that it believes will win. The dataset contains a target feature winner_car with the number of the car that won the race in a level. In the next step the agent observe the next level and takes and new action. If the agent have managed to pick a winning car amongst all cars in a particular level the reward is calculated as car_odds*level_reward_factor. The level_reward factor is 10. If the agent don't manage to pick a winning car the reward is -10. If the agent manages to pick a winning car in all levels of a particular world it gets a world reward which is calculated as the sum of the level rewards of the particular world in which it manages to pick winning cars in all levels of the world. The game is over if the reward reaches -100 or when the agent have played all worlds and levels. Now I need your help with writing code for the RL function that represents the environment. Then after you have responded I will ask you for providing the code for the other RL functions needed to be able to train a RL agent on the dataset we name car_worlds.

In [ ]:
'Below code will create the dataset car_worlds. Can you help me with writing the code for the self.observation_space = spaces...

'In my game the game starts with observing the data for the first level in a world in the game. The observation is a number of features for the level such as the car_number and car_odds of all the cars competing to win in a particular level. From the observation the agent acts by picking either 1, 2 or three cars that it believes will win that level. In the all_data dataset there is a target feature called winner_car that shows the car number of the winning car. When the agent has observed the level it chooses and action by picking 1, 2 or three cars which it believes will win the level. The winner_car feature is used to determine the reward if the agent is able to pick the winning car in a level. If the agent manages to pick the winning car the reward is calculated by taking the car_odds multiplied by 10. The next step is the next level in a world and so the game continues step by step observing one level at a time. can you try to write the code for the step function?


In [197]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 7
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   level_number      60 non-null     int64  
 1   car_number        60 non-null     int64  
 2   wheel_type        60 non-null     object 
 3   motor_endurance   60 non-null     object 
 4   odds              60 non-null     int64  
 5   level_distance    60 non-null     int64  
 6   level_difficulty  60 non-null     object 
 7   winner_probs      60 non-null     float64
 8   winner            60 non-null     int64  
 9   world_id          60 non-null     object 
 10  world_type        60 non-null     object 
dtypes: float64(1), int64(5), object(5)
memory usage: 5.6+ KB


In [6]:
def get_level_df(worlds, current_world_id, current_level_number): # Get the current world id and level number from some attributes
 
 for key, value in worlds.items(): # Check if the key matches the current world id
    if key == current_world_id: # Get the dataframe for the current world
      print(key, current_world_id, current_level_number)
      curr_world_df = value
      print(curr_world_df)
      level_row = curr_world_df[curr_world_df["level_number"] == current_level_number]
      print(level_row)
      cars_data = level_row["cars_data"]
      print(cars_data)
      curr_level_df = cars_data.iloc[0]
      print(level_df)
    else:
      print("world id or level number not in index")
    
    

In [8]:
current_world_id = 'XYW'

level_df_curr = get_level_df(worlds, current_world_id, 3)
level_df_curr

world id or level number not in index
XYW XYW 3
  world_id world_type  level_number  \
0      XYW         V3             1   
1      XYW         V3             2   
2      XYW         V3             3   

                                           cars_data  
0     level_number  car_number wheel_type motor_e...  
1     level_number  car_number wheel_type motor_e...  
2     level_number  car_number wheel_type motor_e...  
  world_id world_type  level_number  \
2      XYW         V3             3   

                                           cars_data  
2     level_number  car_number wheel_type motor_e...  
2       level_number  car_number wheel_type motor_e...
Name: cars_data, dtype: object
   level_number  car_number wheel_type motor_endurance  odds  level_distance  \
0             4           4        red          medium     2            1500   
1             4           1        red            high    23            1500   
2             4           3       blue          medium    28

In [ ]:
##### CUSTOM GYM  

In [31]:
# Stable Baselines only supports tensorflow 1.x for now
#%tensorflow_version 1.x


In [ ]:
!pip install stable-baselines[mpi]==2.10.0

In [94]:
import gym
from gym import spaces

In [9]:
def nums_chosen_random(num_random_nums):
    # Skapa en lista med tal från 0 till 14
    list_car_nums = list(np.random.permutation(np.arange(1, num_cars + 1)))
    print(list_car_nums)
    # Välj antalet tal som ska väljas baserat på den andra åtgärden
    if num_random_nums == 0:
        antal = 1 # Välj ett tal
    elif num_random_nums == 1:
        antal = 2 # Välj två tal
    else:
        antal = 3 # Välj tre tal
    # Använd random.sample() för att välja slumpmässiga tal från listan
    valda_tal = random.sample(list_car_nums, antal)
    # Returnera de valda talen
    return valda_tal



nrn = random.randrange(0,3,1)
print("n random nums: "+str(nrn+1)+" "+str([nrn]))

ncr = nums_chosen_random(nrn)
print("nums chosen: "+str(ncr))



n random nums: 2 [1]
[1, 3, 2, 4]
nums chosen: [1, 2]


In [ ]:

class CustomCarWorldEnv(gym.Env):
    
    metadata = {'render.modes': ['console']} #we don't have a GUI in colab
    
    #define constants for clearer code
    pick_min = 1
    pick_max = 3

    def __init__(self, dataset):
        
        super(CustomCarWorldEnv, self).__init__()
        
        # Load the data
        self.dataset = worlds

        # Define the reward range
        self.reward_range = (-100, float("inf"))

        # Define action space 
        self.action_space = spaces.MultiDiscrete([15, 3]) #Discrete(3)

        # Define obs space
        observation_space = gym.spaces.Box(low=level_df.min(), high=level_df.max(), shape=level_df.shape, dtype=np.float32)

    def step(self, action):

        # Validate the action
        assert self.action_space.contains(action)

        # Update the state according to the game logic
        #self.state = ...

        # Get the current state variables
        world_id = self.state["world_id"]
        world_type = self.state["world_type"]
        level_number = self.state["level_number"]
        level_difficulty = self.state["level_difficulty"]
        level_distance = self.state["level_distance"]
        cars = self.state["cars"]
        
        # Initialize the reward and done flag
        reward = np.random.normal(loc = action, scale = action)
        done = True
            
        info = {}

        return state, reward, done, info

    def reset(self):
        state = 0
        return state
  
    def render(self, mode='console'):
        if mode != 'console':
          raise NotImplementedError()
        pass

    def close(self):
        pass

In [ ]:
# Define the observation space
        self.observation_space = spaces.Dict({
            "world_id": spaces.Discrete(all_data['world_id'].nunique()),
            "world_type": spaces.Discrete(all_data['world_type'].nunique()),
            "level_number": spaces.Discrete(max(all_data['level_number'])),
            "level_difficulty": spaces.Discrete(all_data['level_difficulty'].nunique()),
            "level_distance": spaces.Box(low=min(all_data['level_distance']), high=max(all_data['level_distance']), shape=(1,), dtype=int),
            "cars": spaces.Tuple([
                spaces.Dict({
                    "car_number": spaces.Discrete(all_data['car_number'].nunique()),
                    "car_wheel_type": spaces.Discrete(all_data['wheel_type'].nunique()),
                    "car_motor_endurance": spaces.Discrete(all_data['motor_endurance'].nunique()),
                    "car_odds": spaces.Box(low=min(all_data['odds'], high=max(all_data['odds'], shape=(1,), dtype=int),
                    "winner_probs": spaces.Box(low=min(all_data['winner_probs'], high=max(all_data['winner_probs'], shape=(1,), dtype=float)
                }) for _ in range(max(all_data['car_number'])-1)
            ])
        })

In [ ]:
def get_agent(world_id):
  # create an agent object with some attributes
  agent = {}
  agent["world_id"] = world_id # the world that the agent is playing in
  agent["level_number"] = 1 # the current level that the agent is playing
  agent["cars_data"] = worlds[world_id][0] # the data of the cars in the current level
  agent["picked_cars"] = [] # a list of car numbers that the agent has picked for the current level
  return agent

In [ ]:

def get_state(agent):
  # create a state vector with some features
  state = []
  state.append(agent["level_number"]) # append the level number as a feature
  state.append(agent["cars_data"]["level_distance"].iloc[0]) # append the level distance as a feature
  state.append(agent["cars_data"]["difficulty_level"].iloc[0]) # append the difficulty level as a feature
  
  for i in range(len(agent["cars_data"])): # loop through each car in the current level
    car_features = [] # create a list of features for each car
    car_features.append(agent["cars_data"]["car_number"].iloc[i]) # append the car number as a feature
    car_features.append(agent["cars_data"]["wheel_type"].iloc[i]) # append the wheel type as a feature
    car_features.append(agent["cars_data"]["endurance_level"].iloc[i]) # append the endurance level as a feature
    car_features.append(agent["cars_data"]["odds"].iloc[i]) # append the odds as a feature
    
    if agent["cars_data"]["car_number"].iloc[i] in agent["picked_cars"]: # check if this car is picked by the agent or not
      car_features.append(1) # append 1 if picked 
    else:
      car_features.append(0) # append 0 if not picked
    
    state.extend(car_features) # extend the state vector with these features
  
  return np.array(state) # convert to numpy array and return

In [ ]:
def get_action(agent):
  # create an action vector with some choices
  action = []
  
  if len(agent["picked_cars"]) < 3: # check if the agent can pick more cars or not
    available_cars = agent["cars_data"]["car_number"].tolist() # get a list of available car numbers in the current level
    for car in agent["picked_cars"]: # loop through each car that the agent has already picked
      available_cars.remove(car) # remove it from the available list
    
    picked_car = random.choice(available_cars) # randomly pick one car from the available list
    action.append(picked_car) # append it to the action vector
  
  return np.array(action) # convert to numpy array and return

In [ ]:
def get_reward(agent):
  # calculate reward based on agent's picked cars and level winner
  reward = 0
  
  if len(agent["picked_cars"]) == 3: # check if the agent has picked three cars or not
    level_winner = agent["cars_data"]["winner"].iloc[0] # get the winner car number for the current level
    
    if level_winner in agent["picked_cars"]: # check if the agent has picked the winner car or not
      reward = 1 # give a positive reward of 1
    else:
      reward = -1 # give a negative reward of -1
  
  return reward

In [ ]:
# import some libraries

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# define some constants
NUM_STATES = 30 # the size of the state vector
NUM_ACTIONS = 3 # the number of possible actions (pick one car out of three)
BATCH_SIZE = 32 # the size of the minibatch for training
MEMORY_SIZE = 10000 # the size of the replay memory buffer
LEARNING_RATE = 0.001 # the learning rate for the optimizer
DISCOUNT_FACTOR = 0.95 # the discount factor for future rewards
EPSILON_START = 1.0 # the initial exploration rate
EPSILON_END = 0.01 # the final exploration rate
EPSILON_DECAY = 0.995 # the decay rate for exploration

# create a DQN model using Keras

model = Sequential()
model.add(Dense(64, input_dim=NUM_STATES, activation="relu")) # input layer with state vector as input and 64 hidden units with relu activation function 
model.add(Dense(32, activation="relu")) # hidden layer with 32 hidden units and relu activation function 
model.add(Dense(NUM_ACTIONS, activation="linear")) # output layer with action vector as output and linear activation function 
model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE)) # compile model with mean squared error loss function and Adam optimizer 

# create a replay memory buffer using a deque (double-ended queue)

memory = deque(maxlen=MEMORY_SIZE) 

# initialize epsilon for exploration-exploitation trade-off

epsilon = EPSILON_START 

# loop through each world

for world_id in world_ids:

    # create an agent object using get_agent function
    
    agent = get_agent(world_id)

    # loop through each level
    
    while agent["level_number"] <= LEVELS_PER_WORLD[world_ids.index(world_id)]:
        
        # reset some variables
        
        done = False 
        total_reward = 0 
        
        # get initial state vector using get_state function
        
        state = get_state(agent) 
        
        while not done: 
            
            if np.random.rand() < epsilon: 
                action_index =
 random.randrange(NUM_ACTIONS) 
                action =
 np.zeros(NUM_ACTIONS) 
                action[action_index] =
Received

In [ ]:
# Define parameters
world_reward_factor = 10 # how much extra reward for completing a world
level_reward_factor = 10

# Define reward function
def get_reward(agent_action_vector,
               winning_car_index,
               odds_vector,
               current_level,
               current_world,
               total_reward):
    # Check if agent picked any winning car
    if agent_action_vector[winning_car_index] == 1:
        # Calculate level reward as product of odds times factor
        current_level_reward = odds_vector[winning_car_index] * level_reward_factor
    else:
        # No level reward for picking wrong car
        current_level_reward = -10
    
    # Update total level reward with level reward
    total_level_reward += current_level_reward
    
    # Check if last level of world
    if current_level == num_levels:
        # Check if positive total level reward
        if total_level_reward > 0:
            # Add extra world reward as sum of all rewards
            world_reward = total_level_reward * world_reward_factor
            total_reward += world_reward
    
    return total_reward

In [ ]:
# Define parameters
max_cars = 3 # maximum number of cars to pick in each level
odds_factor = 0.1 # how much odds influence probability of picking

# Define action space function
def get_action(state_vector):
    # Initialize action vector with zeros
    num_cars = len(state_vector)
    action_vector = np.zeros(num_cars)
    
    # Initialize number of picked cars with zero
    num_picked = 0
    
    # Loop over each car in state vector
    for i in range(num_cars):
        # Get odds of car i
        odds_i = state_vector[i][0]
        
        # Calculate probability of picking car i based on odds and factor
        prob_i = odds_i * odds_factor
        
        # Randomly decide whether to pick car i or not based on probability
        pick_i = random.random() < prob_i
        
        # Check if car i is picked and number of picked cars is less than max
        if pick_i and num_picked < max_cars:
            # Set action vector element i to one
            action_vector[i] = 1
            
            # Increment number of picked cars by one
            num_picked += 1
    
    return action_vector

In [ ]:
#define additinal constants

MAX_CARS = 3 # maximum number of cars to pick per level

REWARD_FACTOR = 10 # reward multiplier for picking a winning car



In [ ]:
# initialize variables

state = [0 ,0] # [world ,level]

action = None # chosen cars

total_reward = 0 # cumulative reward

q_table = {} # Q-table for storing state-action values

epsilon = 0.1 # exploration rate

alpha = 0.1 # learning rate

gamma = 0.9 # discount factor


In [ ]:
 define four main components: the agent, the environment, the action space and the reward function2.

In [ ]:
# define reward function

def reward(action):
    # action is a tuple of car numbers
    # winner is a global variable that stores the winning car number for each level
    if winner in action:
        # get the odds for the winning car
        odds = get_odds(winner)
        # multiply by 10
        return odds * 10
    else:
        # no reward if none of the chosen cars wins
        return 0

def world_reward(world):
    # world is a list of rewards for each level in a world
    # check if all rewards are positive
    if all(r > 0 for r in world):
        # sum up all rewards
        return sum(world)
    else:
        # no extra reward otherwise
        return 0

In [ ]:
# define functions

def get_cars(level):

    """Returns a list of car numbers with their odds"""

    num_cars = CARS_PER_LEVEL[level]

    cars = np.arange(1 ,num_cars + 1)

    odds = np.random.uniform(1 ,5 ,num_cars)

    return list(zip(cars ,odds))

def get_winner(cars):

    """Returns a random car number based on their odds"""

    # normalize odds to probabilities
    probs = np.array([o for _ ,o in cars]) / sum(o for _ ,o in cars)

    # sample a car number using probabilities
    winner = np.random.choice([c for c ,_ in cars] ,p=probs)

    return winner

def choose_action(state ,cars):

    """Returns an action using UCB algorithm"""

    global model # linear regression model